## Avaliação Intermediária

In [3]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='figurino',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

In [4]:
db('SELECT * FROM Personagem')

Executando query:
(1, 'Max')
(2, 'Lando')


In [5]:
db('SELECT * FROM Cena')

Executando query:
(1, 1, 5)
(2, 5, 12)


In [6]:
db('SELECT * FROM Roupa')

Executando query:
(1, 'camisa laranja e calça preta jeans')
(2, 'vestido azul escuro')
(3, 'macacão listrado')
(4, 'camiseta e shorts jeans vermelho')
(5, 'vestido longo preto')


In [7]:
db('SELECT * FROM Figurino')

Executando query:
(1, 1, 1, 'Holanda, laranja mecanica')
(1, 2, 1, 'Continuacao da Holanda')
(1, 2, 2, 'Laranja, mas agora alegre ')
(2, 1, 1, 'Azul, um grande lago')
(2, 2, 1, 'Azul, mas agora triste')
(2, 2, 2, 'Azul escuro misturado, agora sereno')
(3, 1, 1, 'Listrado, como uma lagarta')
(3, 2, 1, 'Listrado, mas virou borboleta')
(4, 2, 1, 'Vermelho, como vinho')
(5, 1, 2, 'Preto, noite, escuro')


1)

a) (1,0 pt.) Identificar qual o personagem que tem o figurino com maior número de peças de vestuário.

In [10]:
db('''
SELECT
    nome, COUNT(id_roupa) AS numero_pecas
FROM 
    Figurino
    INNER JOIN Personagem USING (id_personagem)
GROUP BY
    id_personagem
ORDER BY
    numero_pecas DESC
LIMIT
    1
''')

Executando query:
('Max', 7)


c) (1,0 pt.) Determinar quantos minutos do filme, considerando todas as cenas, contem algum personagem vestindo
peças de roupa cuja descrição contém a palavra “jeans” (e.g. “jeans”, “jeans desbotado”, “jaqueta jeans”, “meia
jeans cano longo”, “ceroulas jeans”, “chapéu jeans de cowboy”, etc). Dica: use o operador LIKE para detectar se uma
string aparece dentro de outra string

In [15]:
db('''
SELECT
    SUM(minuto_final - minuto_inicio) as minutos_total
FROM 
    Cena
    INNER JOIN Figurino USING (id_cena)
    INNER JOIN Roupa USING (id_roupa)
WHERE
    Roupa.descricao LIKE "%jeans%"
''')

Executando query:
(Decimal('25'),)


d) (1,0 pt.) Determinar quais peças de roupa nunca são usadas por um personagem de nome “Bob”

In [26]:
db('''
INSERT INTO Roupa (id_roupa, descricao)
VALUES (6, "vestido rosa, tiara rosa"), (7, "camiseta larga e branca")
''')

Executando query:


In [27]:
db('SELECT * FROM Roupa')

Executando query:
(1, 'camisa laranja e calça preta jeans')
(2, 'vestido azul escuro')
(3, 'macacão listrado')
(4, 'camiseta e shorts jeans vermelho')
(5, 'vestido longo preto')
(6, 'vestido rosa, tiara rosa')
(7, 'camiseta larga e branca')


In [30]:
db('''
SELECT
    id_personagem INTO @id_Bob
FROM 
    Personagem
WHERE
    nome = 'Max'
''')

db('SELECT @id_Bob')

Executando query:
Executando query:
(1,)


In [41]:
db('DROP TEMPORARY TABLE IF EXISTS roupas_Bob')

db('''
CREATE TEMPORARY TABLE roupas_Bob
    SELECT
        DISTINCT (id_roupa), Roupa.descricao, id_personagem
    FROM
        Roupa
        INNER JOIN Figurino USING (id_roupa)
    WHERE
        id_personagem = @id_Bob
''')

db('SELECT * FROM roupas_Bob')

Executando query:
Executando query:
Executando query:
(1, 'camisa laranja e calça preta jeans', 1)
(2, 'vestido azul escuro', 1)
(3, 'macacão listrado', 1)
(4, 'camiseta e shorts jeans vermelho', 1)


In [46]:
db('''
SELECT
    id_roupa, Roupa.descricao
FROM
    Roupa
    LEFT OUTER JOIN roupas_Bob USING (id_roupa)
''')

Executando query:
(1, 'camisa laranja e calça preta jeans')
(2, 'vestido azul escuro')
(3, 'macacão listrado')
(4, 'camiseta e shorts jeans vermelho')
(5, 'vestido longo preto')
(6, 'vestido rosa, tiara rosa')
(7, 'camiseta larga e branca')


In [35]:
db('''
SELECT
    DISTINCT (id_roupa), Roupa.descricao
FROM
    Roupa
    LEFT OUTER JOIN Figurino USING (id_roupa)
WHERE
    id_personagem = @id_Bob
''')

Executando query:
(1, 'camisa laranja e calça preta jeans')
(2, 'vestido azul escuro')
(3, 'macacão listrado')
(4, 'camiseta e shorts jeans vermelho')


DEPOIS VOLTA NESSA E PENSA:
    
FAZ UMA TEMP TABLE COM AS ROUPAS QUE O BOB USA. AS QUE NÃO USAR, LEFT OUTER JOIN.

e) (0,5 pts.) Vamos inventar um conceito novo: a assinatura digital de uma peça de roupa é o valor da função
MD5(id_roupa) do MySQL convertido para binário usando a função UNHEX() do MySQL, ou seja: a assinatura de
id_roupa é UNHEX(MD5(id_roupa)). Faça uma tabela temporária contendo o id_roupa e sua respectiva assinatura de
roupa.

In [48]:
db('DROP TEMPORARY TABLE IF EXISTS assinatura_digital_roupa')

db('''
CREATE TEMPORARY TABLE assinatura_digital_roupa
    SELECT
        id_roupa, UNHEX(MD5(id_roupa)) as assinatura_digital
    FROM
        Roupa
''')

db('SELECT * FROM assinatura_digital_roupa')

Executando query:
Executando query:
Executando query:
(1, bytearray(b'\xc4\xcaB8\xa0\xb9#\x82\r\xccP\x9aou\x84\x9b'))
(2, bytearray(b'\xc8\x1er\x8d\x9dL/co\x06\x7f\x89\xcc\x14\x86,'))
(3, bytearray(b'\xec\xcb\xc8~K\\\xe2\xfe(0\x8f\xd9\xf2\xa7\xba\xf3'))
(4, bytearray(b'\xa8\x7f\xf6y\xa2\xf3\xe7\x1d\x91\x81\xa6{uB\x12,'))
(5, bytearray(b'\xe4\xda;\x7f\xbb\xce#E\xd7w+\x06t\xa3\x18\xd5'))
(6, bytearray(b'\x16y\t\x1cZ\x88\x0f\xafo\xb5\xe6\x08~\xb1\xb2\xdc'))
(7, bytearray(b'\x8f\x14\xe4_\xce\xea\x16zZ6\xde\xddK\xea%C'))


f) (1,0 pt.) A assinatura de um figurino é a agregação das respectivas assinaturas de roupa usando a função de
agregação BIT_XOR() do MySQL. Usando a tabela do item (e) (mesmo que você não a tenha feito), construa uma
tabela temporária que lista a assinatura de figurino para cada cena e personagem nela. Ou seja, sua tabela terá as
colunas (id_cena, id_personagem, assinatura de figurino).

In [54]:
db('DROP TEMPORARY TABLE IF EXISTS assinatura_figurino')

db('''
CREATE TEMPORARY TABLE assinatura_figurino
    SELECT 
        id_cena, id_personagem, BIT_XOR(assinatura_digital) as assinatura_fig
    FROM
        assinatura_digital_roupa
        INNER JOIN Figurino USING (id_roupa)
    GROUP BY
        id_cena, id_personagem
''')

db('SELECT * FROM assinatura_figurino')

Executando query:
Executando query:
Executando query:
(1, 1, bytearray(b'\xe0\x1f\xf8\xcbv\xa9\xee\x1fJ\xfa\xa0\xcaQ\xc6\xb8D'))
(2, 1, bytearray(b'H`\x0e\xb2\xd4Z\t\x02\xdb{\x06\xb1$\x84\xaah'))
(2, 2, bytearray(b'\x0c\xd40\xb5=\xf5\x0c\xe1b\xca/\x13\xa3a\x02\xb7'))
(1, 2, bytearray(b'\xe4\xda;\x7f\xbb\xce#E\xd7w+\x06t\xa3\x18\xd5'))


g) (1,0 pt.) Usando a tabela do item (f) (mesmo que você não a tenha feito) determine quais personagens sempre
usam o mesmo figurino. DICA: conte o numero de figurinos distintos que o personagem usou no filme

In [65]:
db('''
SELECT
    id_personagem, (COUNT(assinatura_fig) - COUNT(DISTINCT(assinatura_fig))) as fig_iguais 
FROM
    assinatura_figurino
GROUP BY
    id_personagem
HAVING
    fig_iguais != 0
''')

Executando query:


h) (0,5 pts.) O pessoal de roteiro da produtora gostou da base de dados dos figurinistas, e acha que pode usá-la
diretamente para contabilizar quais personagens aparecem em qual cena. Explique porque isso não vai funcionar
com um contraexemplo.

Usar para contabilizar quais personagens aparecem na cena

Isso não funcionaria. Isso porque um mesmo personagem pode aparecer na mesma cena mais de uma vez usando figurinos diferentes. Assim, por exemplo, se na cena 1 o Bob veste uma camiseta, uma calça e chapéu e ainda na mesma cena ele aparece depois vestindo um macacão e um tênis, continuamos tendo apenas um mesmo personagem na mesma cena - que apenas trocou as vestimentas. Nesse caso, se usássemos os dados dos figurinistas para contar quais personagens aparecem na cena, provavelmente seriam contados 2, quando na verdade é apenas 1.

Escreva a DDL para modificar o banco de
dados e incorporar a informação dos filmes (id_filme, titulo) e da conexão com as suas cenas.

In [ ]:
'''
DROP TABLE IF EXISTS Filmes

CREATE TABLE Filmes(
    id_filme INT NOT NULL,
    titulo VARCHAR(80) NOT NULL,
    PRIMARY KEY (id_filme)
);

CREATE TABLE Filme_Cenas(
    id_filme INT NOT NULL,
    id_cena INT NOT NULL,
    PRIMARY KEY(id_filme, id_cena)
);

ALTER TABLE Filme_Cenas
    ADD CONSTRAINT fk_cena
        FOREIGN KEY (id_cena)
            REFERENCES Cena(id_cena);


ALTER TABLE Filme_Cenas
    ADD CONSTRAINT fk_filme
        FOREIGN KEY (id_filme)
            REFERENCES Filmes(id_filme);




'''

In [2]:
connection.close()